In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

# Checking for bugs

In [ ]:
name_of_evil_file = "20180919_FC_LFP2_cen14_01_cc_01.res"
datapath = "/Users/jepe/scripting/cellpy/dev_data/bugfixing"

In [ ]:
import os
import sys
import time
from pathlib import Path
print(f"running {sys.argv[0]}")

import cellpy
from cellpy import log
from cellpy import cellreader
from cellpy.parameters import prms

prms.Reader.use_cellpy_stat_file = False
#prms.Reader.cycle_mode = "cathode"
prms.Reader.sorted_data = False
log.setup_logging(default_level="INFO", custom_log_dir=os.getcwd())

filename = Path(datapath) / name_of_evil_file

assert os.path.isfile(filename)

In [ ]:
d = cellreader.CellpyData()
d.from_raw(filename)
d.set_mass(0.12)
d.make_step_table()
d.make_summary()

# checking extracting cycles
n = d.get_number_of_cycles()
c = d.get_cycle_numbers()

In [ ]:

current = d.dataset.raw
steps = d.dataset.steps

In [ ]:
plt.close()

In [ ]:
ch.iloc[-1]

In [ ]:
fig, ax = plt.subplots()
for cycle in c:
    print(f"{cycle}", end=".")
    ch, volt_ch = d.get_ccap(cycle=cycle)
    dc, volt_dc = d.get_dcap(cycle=cycle)
    last_point = ch.iloc[-1]
    v = pd.concat([volt_ch, volt_dc])
    cap = pd.concat([ch, last_point-dc])
    
    ax.plot(cap, v)

   

In [ ]:
def extract_dqdv(cell_data, extract="charge", last_cycle=None):
    """Simple wrapper around the cellpy.utils.ica.dqdv function."""

    from cellpy.utils.ica import dqdv
    list_of_cycles = cell_data.get_cycle_numbers()
    if last_cycle is not None:
        list_of_cycles = [c for c in list_of_cycles if c <= int(last_cycle)]
        lprint(f"only processing up to cycle {last_cycle}")
        lprint(f"you have {len(list_of_cycles)} cycles to process")
    out_data = []
    for cycle in list_of_cycles:
        if extract=="charge":
            c, v = cell_data.get_ccap(cycle)
        else:
            c, v = cell_data.get_dcap(cycle)
        if v.any():
            try:
                v, dq = dqdv(v, c)
                v = v.tolist()
                dq = dq.tolist()
            except IndexError or OverflowError as e:
                v = list()
                dq = list()
                print(" -could not process this (cycle %i)" % cycle)
                print(" %s" % e)

            header_x = "dQ cycle_no %i" % cycle
            header_y = "voltage cycle_no %i" % cycle
            dq.insert(0, header_x)
            v.insert(0, header_y)

            out_data.append(v)
            out_data.append(dq)
        else:
            print(f"Empty step encountered for cycle={cycle}")
    return out_data

In [ ]:
o1 = extract_dqdv(d, "discharge")
o2 = extract_dqdv(d, "charge")

In [ ]:
import warnings
warnings.filterwarnings("default")

In [ ]:

from cellpy.utils.ica import dqdv
c, v = d.get_ccap(2)
v2, dq = dqdv(v, c)

In [ ]:
fig2, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
ax1.plot(v2, dq)
ax2.plot(v, c)
ax1.set_ylabel("dqdv")
ax2.set_ylabel("cap")
ax2.set_xlabel("voltage")

In [ ]:
dc, v3 = d.get_dcap(2)
v4, dq2 = dqdv(v3, dc)

In [ ]:
fig3, (ax3, ax4) = plt.subplots(nrows=2, sharex=True)
ax3.plot(v4, dq2)
ax4.plot(v3, dc)
ax3.set_ylabel("dqdv")
ax4.set_ylabel("cap")
ax4.set_xlabel("voltage")

In [ ]:
print(f"NOTE! get_ccap returns {type(v)} while dqdv returns {type(v2)}")
print("NOTE! also important to note that numpy issues a deprecation warning when running",
      "the scipy savgol_filter")

# SUMMARY
The dqdv routine should be improved in the future
- if possible, it should return a dataframe
- if possible, it should be an option to return a dataframe in tidy format with all cycles
- if possible, it should be an option to return a xyyyyyy...y frame (interpolated)